# week 3-1create the dataframe

In [12]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

google "PostalCode, Borough, and Neighborhood toronto wiki" and get url

In [3]:
website_url = requests.get('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M').text

In [4]:
soup = BeautifulSoup(website_url,'lxml')

In [6]:
table_contents = []
table = soup.find('table')

In [8]:
table

<table cellpadding="2" cellspacing="0" rules="all" style="width:100%; border-collapse:collapse; border:1px solid #ccc;">
<tbody><tr>
<td style="width:11%; vertical-align:top; color:#ccc;">
<p><b>M1A</b><br/><span style="font-size:85%;"><i>Not assigned</i></span>
</p>
</td>
<td style="width:11%; vertical-align:top; color:#ccc;">
<p><b>M2A</b><br/><span style="font-size:85%;"><i>Not assigned</i></span>
</p>
</td>
<td style="width:11%; vertical-align:top;">
<p><b>M3A</b><br/><span style="font-size:85%;"><a href="/wiki/North_York" title="North York">North York</a><br/>(<a href="/wiki/Parkwoods" title="Parkwoods">Parkwoods</a>)</span>
</p>
</td>
<td style="width:11%; vertical-align:top;">
<p><b>M4A</b><br/><span style="font-size:85%;"><a href="/wiki/North_York" title="North York">North York</a><br/>(<a href="/wiki/Victoria_Village" title="Victoria Village">Victoria Village</a>)</span>
</p>
</td>
<td style="width:11%; vertical-align:top;">
<p><b>M5A</b><br/><span style="font-size:85%;"><a hr

In [9]:
for row in table.findAll('td'):
    cell = {}
    if row.span.text=='Not assigned':
        pass
    else:
        cell['PostalCode'] = row.p.text[:3]
        cell['Borough'] = (row.span.text).split('(')[0]
        cell['Neighborhood'] = (((((row.span.text).split('(')[1]).strip(')')).replace(' /',',')).replace(')',' ')).strip(' ')
        table_contents.append(cell)

In [10]:
table_contents

[{'PostalCode': 'M3A', 'Borough': 'North York', 'Neighborhood': 'Parkwoods'},
 {'PostalCode': 'M4A',
  'Borough': 'North York',
  'Neighborhood': 'Victoria Village'},
 {'PostalCode': 'M5A',
  'Borough': 'Downtown Toronto',
  'Neighborhood': 'Regent Park, Harbourfront'},
 {'PostalCode': 'M6A',
  'Borough': 'North York',
  'Neighborhood': 'Lawrence Manor, Lawrence Heights'},
 {'PostalCode': 'M7A',
  'Borough': "Queen's Park",
  'Neighborhood': 'Ontario Provincial Government'},
 {'PostalCode': 'M9A',
  'Borough': 'Etobicoke',
  'Neighborhood': 'Islington Avenue'},
 {'PostalCode': 'M1B',
  'Borough': 'Scarborough',
  'Neighborhood': 'Malvern, Rouge'},
 {'PostalCode': 'M3B',
  'Borough': 'North York',
  'Neighborhood': 'Don Mills North'},
 {'PostalCode': 'M4B',
  'Borough': 'East York',
  'Neighborhood': 'Parkview Hill, Woodbine Gardens'},
 {'PostalCode': 'M5B',
  'Borough': 'Downtown Toronto',
  'Neighborhood': 'Garden District, Ryerson'},
 {'PostalCode': 'M6B', 'Borough': 'North York', 'N

In [13]:
df=pd.DataFrame(table_contents)
df['Borough']=df['Borough'].replace({'Downtown TorontoStn A PO Boxes25 The Esplanade':'Downtown Toronto Stn A',
                                             'East TorontoBusiness reply mail Processing Centre969 Eastern':'East Toronto Business',
                                             'EtobicokeNorthwest':'Etobicoke Northwest','East YorkEast Toronto':'East York/East Toronto',
                                             'MississaugaCanada Post Gateway Processing Centre':'Mississauga'})

In [14]:
df

,PostalCode,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Regent Park, Harbourfront"
3,M6A,North York,"Lawrence Manor, Lawrence Heights"
4,M7A,Queen's Park,Ontario Provincial Government
...,...,...,...
98,M8X,Etobicoke,"The Kingsway, Montgomery Road, Old Mill North"
99,M4Y,Downtown Toronto,Church and Wellesley
100,M7Y,East Toronto Business,Enclave of M4L
101,M8Y,Etobicoke,"Old Mill South, King's Mill Park, Sunnylea, Hu..."


In [15]:
df.shape

(103, 3)

# week 3-2 get the latitude and the longitude

In [21]:
import geocoder # import geocoder

# initialize your variable to None
lat_lng_coords = None

# loop until you get the coordinates
while(lat_lng_coords is None):
  g = geocoder.google('{}, Toronto, Ontario'.format(postal_code))
  lat_lng_coords = g.latlng

latitude = lat_lng_coords[0]
longitude = lat_lng_coords[1]

NameError: name 'postal_code' is not defined

our data is from [GeoSpatial Dataset](https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-DS0701EN-SkillsNetwork/labs_v1/Geospatial_Coordinates.csv)

In [22]:
geospatial_data = pd.read_csv("https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-DS0701EN-SkillsNetwork/labs_v1/Geospatial_Coordinates.csv")

In [26]:
geospatial_data.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [29]:
Toronto_df = pd.merge(
    df, geospatial_data[["Postal Code","Latitude", "Longitude"]],
    left_on = "PostalCode", right_on="Postal Code")
Toronto_df

,PostalCode,Borough,Neighborhood,Postal Code,Latitude,Longitude
0,M3A,North York,Parkwoods,M3A,43.753259,-79.329656
1,M4A,North York,Victoria Village,M4A,43.725882,-79.315572
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",M5A,43.654260,-79.360636
3,M6A,North York,"Lawrence Manor, Lawrence Heights",M6A,43.718518,-79.464763
4,M7A,Queen's Park,Ontario Provincial Government,M7A,43.662301,-79.389494
...,...,...,...,...,...,...
98,M8X,Etobicoke,"The Kingsway, Montgomery Road, Old Mill North",M8X,43.653654,-79.506944
99,M4Y,Downtown Toronto,Church and Wellesley,M4Y,43.665860,-79.383160
100,M7Y,East Toronto Business,Enclave of M4L,M7Y,43.662744,-79.321558
101,M8Y,Etobicoke,"Old Mill South, King's Mill Park, Sunnylea, Hu...",M8Y,43.636258,-79.498509


# week 3-3 Explore and cluster the neighborhoods in Toronto.

In [30]:
Toronto_df_for_cluster = Toronto_df[["Borough", "Latitude", "Longitude"]]

In [35]:
from sklearn import cluster, datasets, metrics
from sklearn.cluster import KMeans
import matplotlib.pyplot as plt

In [39]:
n = 6
datalist = [pd.DataFrame() for _ in range(n)]
kmeans = KMeans(n_clusters = n, init = 'k-means++', max_iter = 300, n_init = 10, random_state = 0)
pred_y = kmeans.fit_predict(Toronto_df_for_cluster[["Latitude", "Longitude"]])
for i in range(0, n):
    datalist[i] = Toronto_df_for_cluster.loc[pred_y == i, ['Borough', 'Latitude', 'Longitude']]

In [40]:
print(datalist[0])

                    Borough   Latitude  Longitude
0                North York  43.753259 -79.329656
1                North York  43.725882 -79.315572
8                 East York  43.706397 -79.309937
13               North York  43.725900 -79.340923
14                East York  43.695344 -79.318389
19             East Toronto  43.676357 -79.293031
23                East York  43.709060 -79.363452
29                East York  43.705369 -79.349372
35   East York/East Toronto  43.685347 -79.338106
41             East Toronto  43.679557 -79.352188
44              Scarborough  43.711112 -79.284577
47             East Toronto  43.668999 -79.315572
54             East Toronto  43.659526 -79.340923
58              Scarborough  43.692657 -79.264848
71              Scarborough  43.750071 -79.295849
100   East Toronto Business  43.662744 -79.321558


In [41]:
print(datalist[1])

                 Borough   Latitude  Longitude
5              Etobicoke  43.667856 -79.532242
11             Etobicoke  43.650943 -79.554724
17             Etobicoke  43.643515 -79.577201
70             Etobicoke  43.696319 -79.532242
76           Mississauga  43.636966 -79.615819
77             Etobicoke  43.688905 -79.554724
88             Etobicoke  43.605647 -79.501321
89             Etobicoke  43.739416 -79.588437
93             Etobicoke  43.602414 -79.543484
94   Etobicoke Northwest  43.706748 -79.594054
98             Etobicoke  43.653654 -79.506944
101            Etobicoke  43.636258 -79.498509
102            Etobicoke  43.628841 -79.520999


In [42]:
print(datalist[2])

        Borough   Latitude  Longitude
6   Scarborough  43.806686 -79.194353
12  Scarborough  43.784535 -79.160497
18  Scarborough  43.763573 -79.188711
22  Scarborough  43.770992 -79.216917
26  Scarborough  43.773136 -79.239476
32  Scarborough  43.744734 -79.239476
38  Scarborough  43.727929 -79.262029
51  Scarborough  43.716316 -79.239476
65  Scarborough  43.757410 -79.273304
78  Scarborough  43.794200 -79.262029
82  Scarborough  43.781638 -79.304302
85  Scarborough  43.815252 -79.284577
95  Scarborough  43.836125 -79.205636


In [43]:
print(datalist[3])

            Borough   Latitude  Longitude
7        North York  43.745906 -79.352188
27       North York  43.803762 -79.363452
28       North York  43.754328 -79.442259
33       North York  43.778517 -79.346556
39       North York  43.786947 -79.385975
45       North York  43.757490 -79.374714
52       North York  43.789053 -79.408493
55       North York  43.733283 -79.419750
59       North York  43.770120 -79.408493
61  Central Toronto  43.728020 -79.388790
66       North York  43.752758 -79.400049
72       North York  43.782736 -79.442259
90      Scarborough  43.799525 -79.318389


In [44]:
print(datalist[4])

         Borough   Latitude  Longitude
3     North York  43.718518 -79.464763
10    North York  43.709577 -79.445073
21          York  43.689026 -79.453512
34    North York  43.767980 -79.487262
40    North York  43.737473 -79.464763
46    North York  43.739015 -79.506944
49    North York  43.713756 -79.490074
50    North York  43.756303 -79.565963
53    North York  43.728496 -79.495697
56          York  43.691116 -79.476013
57    North York  43.724766 -79.532242
60    North York  43.761631 -79.520999
63          York  43.673185 -79.487262
64          York  43.706876 -79.518188
69  West Toronto  43.661608 -79.464763
81  West Toronto  43.651571 -79.484450


In [45]:
print(datalist[5])

                   Borough   Latitude  Longitude
2         Downtown Toronto  43.654260 -79.360636
4             Queen's Park  43.662301 -79.389494
9         Downtown Toronto  43.657162 -79.378937
15        Downtown Toronto  43.651494 -79.375418
16                    York  43.693781 -79.428191
20        Downtown Toronto  43.644771 -79.373306
24        Downtown Toronto  43.657952 -79.387383
25        Downtown Toronto  43.669542 -79.422564
30        Downtown Toronto  43.650571 -79.384568
31            West Toronto  43.669005 -79.442259
36        Downtown Toronto  43.640816 -79.381752
37            West Toronto  43.647927 -79.419750
42        Downtown Toronto  43.647177 -79.381576
43            West Toronto  43.636847 -79.428191
48        Downtown Toronto  43.648198 -79.379817
62         Central Toronto  43.711695 -79.416936
67         Central Toronto  43.712751 -79.390197
68         Central Toronto  43.696948 -79.411307
73         Central Toronto  43.715383 -79.405678
74         Central T

In [55]:
group1 = datalist[0]
group1["group"] = 1
group2 = datalist[1]
group2["group"] = 2
group3 = datalist[2]
group3["group"] = 3
group4 = datalist[3]
group4["group"] = 4
group5 = datalist[4]
group5["group"] = 5
group6 = datalist[5]
group6["group"] = 6

In [60]:
grouped_data = pd.concat([group2, group3, group4, group5, group6])

In [48]:
import plotly.graph_objects as go

In [64]:
fig = go.Figure(data=go.Scattergeo(
        lon = grouped_data['Longitude'],
        lat = grouped_data['Latitude'],
        text = grouped_data['Borough'],
        mode = 'markers',
        marker_color = grouped_data['group'],
        ))

fig.update_layout(
        title = 'neighborhoods in Toronto',
        geo_scope='north america',
    )
fig.show()